In [1]:
from bs4 import BeautifulSoup
import pandas as pd 
import numpy as np 

In [2]:
import re

In [3]:
from datetime import datetime

In [49]:
race = ['Sunderland', datetime.strptime("04Apr19", "%d%b%y"), 'A3', '450']

In [50]:
with open("dog.html", "r") as f_:
    result_page = BeautifulSoup(f_, "html.parser")

In [64]:
def normalize(row, t_):    
    try: 
        if t_ == "text":
            return row.text
        if t_ == "date":
            s = row.text.replace(" ", "")
            s = datetime.strptime(s, "%d%b%y")
            return s
        if t_ == "distance":
            return int(row.text.replace("m", ""))
        if t_ == "only_digits":
            return int(re.sub("\D", "", row.text))
        if t_ == "bends":
            row = row.text.replace("-", "")
            return np.average(np.array(list(row)).astype(int))        
        if t_ == "remarks":
            return row.text.lower().split(",") 
        if t_ == "float":
            return np.float(row.text)
        if t_ == "int":
            return np.int(row.text)
    except Exception as e:
        return "NaN"

In [65]:
stats = list()
for tr in result_page.find("table", {"id":"sortableTable"}).find("tbody").find_all("tr", class_="row"):
    c = tr.find_all("td")
    stats.append([
        normalize(c[0],  "date"),
        normalize(c[1],  "text"),
        normalize(c[2],  "distance"),
        normalize(c[3],  "only_digits"),
        normalize(c[4],  "float"),
        normalize(c[5],  "bends"),
        normalize(c[9],  "remarks"),
        normalize(c[10], "float"),
        normalize(c[11], "float"),
        normalize(c[12], "float"),
        normalize(c[14], "text"),
        normalize(c[15], "float"),
    ])

In [66]:
stats = pd.DataFrame(stats, columns=[
    "date", 
    "local", 
    "distance", 
    "trap", 
    "split", 
    "bends", 
    "position",
    "remarks",
    "win_time",
    "weight",
    "grade",
    "cal_time"
])

In [67]:
stats.head()

,date,local,distance,trap,split,bends,position,remarks,win_time,weight,grade,cal_time
0,2019-05-05,Sland,450,2,5.11,3.00,[clrrn],27.87,NaN,33.1,A3,28.13
1,2019-04-30,Sland,450,2,5.13,2.75,[crd1],27.90,10,33.2,A3,28.32
2,2019-04-21,Sland,450,2,5.1,3.00,[bmp1],27.78,10,33.6,A3,28.20
3,2019-04-12,Sland,450,2,5.15,4.75,[crd1],27.63,NaN,32.7,A3,28.50
4,2019-04-04,Sland,450,2,5.17,1.25,[ld1],28.24,10,32.2,A3,28.34


In [68]:
datetime.now()

datetime.datetime(2019, 5, 11, 18, 40, 31, 274316)

In [81]:
stats = stats[stats["date"] < race[1]]

In [82]:
stats_grade = stats[stats["grade"] == race[2]]

In [85]:
stats_grade

,date,local,distance,trap,split,bends,position,remarks,win_time,weight,grade,cal_time
5,2019-03-31,Sland,450,2,5.18,3.00,[clrrn],27.98,NaN,32.9,A3,28.13
6,2019-03-22,Sland,450,2,5.13,2.75,[crd1],27.85,NaN,32.8,A3,28.52
7,2019-03-17,Sland,450,2,5.04,2.25,[bmp1],27.74,-20,32.9,A3,27.83
10,2019-01-16,Sland,450,2,5.15,4.25,[bmp3],27.62,NaN,32.4,A3,28.14
11,2019-01-06,Sland,450,2,5.08,4.25,[bmp3],27.89,NaN,33.1,A3,28.29
